In [ ]:
from google.cloud import bigquery
from google.colab import drive
from google.oauth2 import service_account
import json, os
import pandas as pd 
import seaborn as sns

In [ ]:
# Google Cloud services
gcp_service_account_credentials_json_filename = 'epfl-course-f41b0ed796f9.json' #need to upload the json credential files to the root directory of the google colab files
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = gcp_service_account_credentials_json_filename
credentials = service_account.Credentials.from_service_account_file(gcp_service_account_credentials_json_filename, scopes=['https://www.googleapis.com/auth/bigquery', 'https://www.googleapis.com/auth/drive'])
project_id = 'epfl-course'
bigquery_client = bigquery.Client(credentials=credentials, project=project_id)
bigquery_client = bigquery.Client()

In [ ]:
#import sample data
iris = sns.load_dataset('iris')

In [ ]:
#try to upload to bq
table_id = "epfl-course.datavizcourse.sample_data"
job_config = bigquery.LoadJobConfig()

job = bigquery_client.load_table_from_dataframe(
    iris, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.


In [ ]:
table = bigquery_client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

In [ ]:
#get the content from google drive
path = r"/content/drive/MyDrive/dataviz/data/"
drive.mount('/content/drive')
file_list=os.listdir(path)

In [ ]:
event_files = []
for f in file_list:
  if 'event' in f:
    event_files.append(f)

print('We have ' + str(len(event_files)) + ' event files')

In [ ]:
#get the sizes:
sizes = {}
for e in event_files:
  sizes[e] = os.path.getsize(path + e)

Try to simulate the behavior without importing

In [ ]:
sorted_names = [x[0] for x in sorted(sizes.items(), key=lambda item: int(item[1]))][3:]
sorted_sizes = [x[1] for x in sorted(sizes.items(), key=lambda item: int(item[1]))][3:]
sorted_sizes_giga = [ x / 10**9 for x in sorted_sizes]
sum(sorted_sizes_giga)

In [ ]:
#read a table
while len(sorted_names) > 1:
  big_one = pd.read_csv(path+event_files[0])
  sorted_names.pop(0)
  sorted_sizes.pop(0)
  sorted_sizes_giga.pop(0)

  #create a 1 gb table
  giga_count = 0
  for name, size, size_giga in zip(sorted_names, sorted_sizes, sorted_sizes_giga):
    #check the total lenght with following element
    giga_count += size_giga
    if giga_count < 0.35:
      #include following element
     # new_df = pd.read_csv(path+name)
      #big_one = pd.concat([big_one, new_df])

      #delete element from to add list
      sorted_names.pop(0)
      sorted_sizes.pop(0)
      sorted_sizes_giga.pop(0)
      print('left:', len(sorted_names), 'giga', giga_count)
    else:
      break

  #add the table to bq
  print('#############################################')
  print('fai qualcosa con bq')

Real deal

In [ ]:
sorted_names = [x[0] for x in sorted(sizes.items(), key=lambda item: int(item[1]))][3:]
sorted_sizes = [x[1] for x in sorted(sizes.items(), key=lambda item: int(item[1]))][3:]
sorted_sizes_giga = [ x / 10**9 for x in sorted_sizes]
sum(sorted_sizes_giga)

In [ ]:
big_one = pd.read_csv(path+event_files[10])
big_one.head()

In [ ]:
# big_one['individual_local_identifier'] = big_one.apply(lambda x: str(x['individual_local_identifier']) , axis = 1)

In [ ]:
# big_one['individual_local_identifier'] = big_one['individual_local_identifier'].astype(str)
# big_one['individual_taxon_canonical_name'] = big_one['individual_taxon_canonical_name'].astype(str)

In [ ]:
# table_id = "epfl-course.datavizcourse.temp"
# job_config = bigquery.LoadJobConfig()

# job = bigquery_client.load_table_from_dataframe(
    # big_one, table_id, job_config=job_config
# )  # Make an API request.
# job.result()  # Wait for the job to complete.

In [ ]:
unprocessed = []
#read a table
while len(sorted_names) > 1:
  big_one = pd.read_csv(path+event_files[0])
  giga_count = sorted_sizes_giga[0]

  sorted_names.pop(0)
  sorted_sizes.pop(0)
  sorted_sizes_giga.pop(0)

  #create a 1 gb table
  giga_count = 0
  temp = []
  for name, size, size_giga in zip(sorted_names, sorted_sizes, sorted_sizes_giga):
    #check the total lenght with following element
    giga_count += size_giga
    if giga_count < 0.10:
      #include following element
      new_df = pd.read_csv(path+name)
      big_one = pd.concat([big_one, new_df])

      #delete element from to add list
      sorted_names.pop(0)
      sorted_sizes.pop(0)
      sorted_sizes_giga.pop(0)
      print('left:', len(sorted_names), 'giga', giga_count)

      temp += [name]
    else:
      break

  try:
    big_one['individual_local_identifier'] = big_one['individual_local_identifier'].astype(str)
    big_one['individual_taxon_canonical_name'] = big_one['individual_taxon_canonical_name'].astype(str)

    #add the table to bq
    table_id = "epfl-course.datavizcourse.tot_events_2"
    job_config = bigquery.LoadJobConfig()

    job = bigquery_client.load_table_from_dataframe(
        big_one, table_id, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.
  except Exception as e: 
    print(e)

In [ ]:
# to delete 
#read a table
while len(sorted_names) > 1:
  big_one = pd.read_csv(path+event_files[0])
  sorted_names.pop(0)
  sorted_sizes.pop(0)
  sorted_sizes_giga.pop(0)

  #create a 1 gb table
  giga_count = 0
  for name, size, size_giga in zip(sorted_names, sorted_sizes, sorted_sizes_giga):
    #check the total lenght with following element
    giga_count += size_giga
    if giga_count < 0.35:
      #include following element
      new_df = pd.read_csv(path+name)
      big_one = pd.concat([big_one, new_df])

      #delete element from to add list
      sorted_names.pop(0)
      sorted_sizes.pop(0)
      sorted_sizes_giga.pop(0)
      print('left:', len(sorted_names), 'giga', giga_count)
    else:
      break

  #add the table to bq
  table_id = "epfl-course.ada_project.tot_events"
  job_config = bigquery.LoadJobConfig()

  job = bigquery_client.load_table_from_dataframe(
      iris, table_id, job_config=job_config
  )  # Make an API request.
  job.result()  # Wait for the job to complete.

In [ ]:
#read a table
while len(sorted_names) > 1:
  big_one = pd.read_csv(path+event_files[0])
  sorted_names.pop(0)
  sorted_sizes.pop(0)
  sorted_sizes_giga.pop(0)

  #create a 1 gb table
  giga_count = 0
  for name, size, size_giga in zip(sorted_names, sorted_sizes, sorted_sizes_giga):
    #check the total lenght with following element
    giga_count += size_giga
    if giga_count < 0.35:
      #include following element
      new_df = pd.read_csv(path+name)
      big_one = pd.concat([big_one, new_df])

      #delete element from to add list
      sorted_names.pop(0)
      sorted_sizes.pop(0)
      sorted_sizes_giga.pop(0)
      print('left:', len(sorted_names), 'giga', giga_count)

  #add the table to bq
  table_id = "epfl-course.ada_project.tot_events"
  job_config = bigquery.LoadJobConfig()

  job = bigquery_client.load_table_from_dataframe(
      iris, table_id, job_config=job_config
  )  # Make an API request.
  job.result()  # Wait for the job to complete.

In [ ]:
big_one = pd.read_csv(path+'studies.csv.gz')

In [ ]:
big_one.head()

In [ ]:
#add the table to bq
table_id = "epfl-course.datavizcourse.studies_names"
job_config = bigquery.LoadJobConfig()

job = bigquery_client.load_table_from_dataframe(
    big_one, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.